In [44]:
import loadmnist as lm
import numpy as np
from contextlib import contextmanager
import time as time
from keras.utils import np_utils

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')


with timer("load MNIST and data preparation"):
    x,y = lm.load_mnist()
    x_train,x_test = x[:60000].astype('float')/255,x[60000:].astype('float')/255
    y_train,y_test = np_utils.to_categorical(y[:60000]),np_utils.to_categorical(y[60000:])            
    shuffle_index = np.random.permutation(60000)
    x_train,y_train = x_train[shuffle_index],y_train[shuffle_index]

import tensorflow as tf
import tensorflow.contrib.keras as keras

EPOCH = 20
BATCH_SIZE = 128

with timer("make model"):
    Sequential = keras.models.Sequential
    Dense = keras.layers.Dense
    Dropout = keras.layers.Dropout
    backend = keras.backend

    model = Sequential()

    model.add(Dense(512,activation='relu',input_shape=(784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10,activation='softmax'))
    
    model.summary()

    model.compile(loss = 'categorical_crossentropy',
                  metrics = ['accuracy'],
                  optimizer='Adam'
                 )

with timer("fitting"):
    model.fit(x_train,
              y_train,
              batch_size=BATCH_SIZE,
              epochs=EPOCH,
              verbose=1,
              validation_data=(x_test,y_test)
             )

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss :', score[0])
print('Test accuracy :', score[1])

[load MNIST and data preparation] done in 3 s
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________
[make model] done in 1 s
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
 307

KeyboardInterrupt: 

In [43]:
json_string = model.to_json()
open('mnist_model.json', 'w').write(json_string)
yaml_string = model.to_yaml()
open('mnist_model.yaml', 'w').write(yaml_string)
model.save_weights('mnist_model.hdf5')